# Red de empresas - Patrones adjudicación
__Queletzú Paulina Aspra Polo y Mariana Esther Martínez Sánchez__

1. Limpieza por sexenio
* Colaboraciones en contratos
* __Patrones adjudicación__


* ¿Cuántos contratos y de que monto se dieron por cada tipo de adjudicación?
    * Sexenio y Año
    * Absoluto y normalizado
* ¿Cuántas empresas contrata cada institucion? dist
* ¿Qué monto asigna por tipo de contrato cada institucion? barra
* ¿Para cuántas instituciones trabaja una empresa? dist
* ¿Qué monto recibe por tipo de adjudicación una empresa? barra
* ¿Qué empresas reciben un solo gran contrato? barra
* Lista puede con todas
* Lista favoritas
* ¿Son las mismas empresas populares a lo largo del tiempo? Venn
    * Todas
    * Puede con todas
    * Las favoritas


## Cargar datos